In [22]:
#%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation, cm
from IPython.display import HTML

In [23]:
num = 11
x = np.linspace(-50, 50, num)
y = np.linspace(-50, 50, num)
x, y = np.meshgrid(x, y)

In [24]:
z = x ** 2 + y ** 2

In [25]:
idx1 = np.array([[1 ,2], [2, 3]])
print(tuple(zip(*idx1)))
z[tuple(zip(*idx1))]

((1, 2), (2, 3))


array([ 2500.,  1300.])

In [26]:
z

array([[ 5000.,  4100.,  3400.,  2900.,  2600.,  2500.,  2600.,  2900.,
         3400.,  4100.,  5000.],
       [ 4100.,  3200.,  2500.,  2000.,  1700.,  1600.,  1700.,  2000.,
         2500.,  3200.,  4100.],
       [ 3400.,  2500.,  1800.,  1300.,  1000.,   900.,  1000.,  1300.,
         1800.,  2500.,  3400.],
       [ 2900.,  2000.,  1300.,   800.,   500.,   400.,   500.,   800.,
         1300.,  2000.,  2900.],
       [ 2600.,  1700.,  1000.,   500.,   200.,   100.,   200.,   500.,
         1000.,  1700.,  2600.],
       [ 2500.,  1600.,   900.,   400.,   100.,     0.,   100.,   400.,
          900.,  1600.,  2500.],
       [ 2600.,  1700.,  1000.,   500.,   200.,   100.,   200.,   500.,
         1000.,  1700.,  2600.],
       [ 2900.,  2000.,  1300.,   800.,   500.,   400.,   500.,   800.,
         1300.,  2000.,  2900.],
       [ 3400.,  2500.,  1800.,  1300.,  1000.,   900.,  1000.,  1300.,
         1800.,  2500.,  3400.],
       [ 4100.,  3200.,  2500.,  2000.,  1700.,  1600.,

In [41]:
def maximize(f, point):
    return f[tuple(point)]

def minimize(f, point):
    return -f[tuple(point)]

In [42]:
def starting_point(shape):
    return np.array([np.random.randint(i) for i in shape])

In [43]:
#neighbor functions 

def l_inf_neighbors(point, shape):
    a = np.arange(-1, 2)
    ns = (point + np.stack(np.meshgrid(*[a for i in range(len(point))]), axis=-1)).reshape(-1, len(point))
    gt = lambda n: np.all(np.less(n, shape)) and np.all(np.greater(n, -1)) and not np.array_equal(n, point)
    return ns[[gt(n) for n in ns]]
    
# l_inf_neighbors(np.array([3, 4]), (5, 5))

def random_neighbors(number, point, shape):
    return np.hstack([np.random.randint(i, size=(number, 1)) for i in shape])   

# random_neighbors(10, [1,2,3,4], (34,4,4))

In [46]:
def greedy(f, fitness, neighbors_function):
    current = starting_point(f.shape)
    better = True
    while better:
        better = False
        for neighbor in neighbors_function(current, f.shape):
            neighbor_value = f[tuple(neighbor)]
            if fitness(f, neighbor) > fitness(f, current):# and neighbor_value != current_value:
                yield neighbor
                current = neighbor
                current_value = neighbor_value
                better = True
                break

In [53]:
for i in greedy(z, minimize, l_inf_neighbors):
    print i

[2 6]
[2 5]
[3 4]
[4 4]
[5 4]
[5 5]


In [31]:
#simmulated annealing: make a random move with a probability p proportional to improvement

def get_temperature(temperature_0, decay_factor, step):
    return temperature_0 * decay_factor ** step

def nat_to_exp(q):
    max_q = max(0.0, np.max(q))
    rebased_q = q - max_q
    return np.exp(rebased_q - np.logaddexp(-max_q, np.logaddexp.reduce(rebased_q)))


def move_prob(f, current, neighbor, temperature):
    d_f = f[tuple(current)] - f[tuple(neighbor)]
    return nat_to_exp(d_f / temperature)
#     return 1 / (1 + np.exp(- d_f / temperature))
    
def simulated_annealing(f, temperature_0, decay_factor, steps, neighbors_function):
    current = starting_point(f.shape)
    yield current
    for step in range(steps):
        temperature = get_temperature(temperature_0, decay_factor, step)
        neighbor = neighbors_function(1, current, f.shape).reshape(-1)
        if move_prob(f, current, neighbor, temperature) > np.random.random_sample():
            current = neighbor
            yield current
            
            
        
temperature_0 = 10
decay_factor = 0.99
steps = 1000

for i in simulated_annealing(z, temperature_0, decay_factor, steps, random_neighbors):
    print(i)
        
        

[8 4]
[6 5]
[4 5]
[5 5]
[5 5]
[5 5]


In [32]:
from collections import deque 

x = deque([], 5)
x.append(1)
x.append(2)
x.append(3)
x.append(4)
x.append(5)
x.append(6)
print(1 in x)
for i in x:
    print(i)


False
2
3
4
5
6


In [77]:
def tabu(f, fitness, neighbors_function, max_tabu_size, steps):
    best = tuple(starting_point(f.shape))
    yield best
    candidate = best
    tabu_list = deque([], max_tabu_size)
    tabu_list.append(best)
    for step in range(steps):
        neighbors = neighbors_function(candidate, f.shape)
        candidate = tuple(neighbors[0])
        for neighbor in neighbors:
            neighbor =  tuple(neighbor)
            if (not candidate in tabu_list) and (fitness(f, neighbor) > fitness(f, candidate)):
                candidate = neighbor
        
        if fitness(f, candidate) > fitness(f, best):
            best = candidate
            yield best

        
        tabu_list.append(candidate)

for i in tabu(z, minimize, l_inf_neighbors, 5, 100):
    print(i)

(0, 5)
(1, 5)
(2, 5)
(3, 5)
(4, 5)
(5, 5)


In [78]:
fig = plt.figure()
Axes3D(fig).plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [13]:
# fig = plt.figure()

# ax = Axes3D(fig)
# surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    
# def init():
#     ax.view_init(elev=10., azim=0)
#     return (surf,)

# def animate(i):
#     ax.view_init(elev=10., azim=i)
#     return (surf,)

# matplotlib.rcParams['animation.writer'] = 'avconv'
# anim = animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=20, blit=True)
# HTML(anim.to_html5_video())